### Run me first

In [1]:
import pandas as pd
import sqlite3
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

%matplotlib inline
%reload_ext autoreload
%autoreload 2

con = sqlite3.connect("collca.db")
cur = con.cursor()
con2 = sqlite3.connect('RobynDatabase.db')
cur2 = con2.cursor()
#for row in cur.execute('select * from khipu_main'):
    #print(row)

In [9]:
from pathlib import Path  



table_names = ['ascher_cord_color','archive_dc','beginning_dc','cord','cord_cluster','khipu_main','knot','knot_cluster','primary_cord','cord_notes','khipu_notes','pcord_notes','canutito']
for tb in table_names:
    filepath = Path(tb+'.csv')  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    df = pd.read_sql_query('select * from '+tb,con)
    df.to_csv(filepath)

In [1]:
khipuNamesDF = pd.read_sql_query("select KHIPU_ID,INVESTIGATOR_NUM,orig_inv_num,PROVENANCE,REGION,ARCHIVE_NUM,MUSEUM_NAME,MUSEUM_NUM from khipu_main",con)

khipuMeta = khipuNamesDF.copy()
khipuMeta = khipuMeta.set_index("KHIPU_ID")
khipuMeta = khipuMeta.drop(996) #ID 996 is my test khipu for the drawing diagrams
khipuMeta['ARCHIVE']=''
khipuMeta['PCORD_LENGTH']=''
khipuMeta['PCORD_TERM']=''
khipuMeta['PCORD_PLY']=''
khipuMeta['PCORD_COLORTYPE']=''
khipuMeta['PCORD_FULL_COLOR']=''
khipuMeta['PCORD_ID']=''
#khipuMeta['PCORD_COLOR_COUNT']='' is >1 if colortype contains '/'
khipuMeta['PRIM_BEG']=''
khipuMeta['PCORD_HAS_KNOTS']=''
khipuMeta['PCORD_HAS_MARKERS']=''
khipuMeta['SERIATED']=''
khipuMeta['BANDED']=''
#khipuMeta['COLONIAL']=''
#khipuMeta['INCA']=''
#khipuMeta['WARI']=''
khipuMeta['COUNT_PENDANTS']=''
khipuMeta['COUNT_TOPS']=''
khipuMeta['COUNT_SUBS']=''
khipuMeta['COUNT_DISTINCT_COLORS']=''
khipuMeta['DISTINCT_CORD_PLY']=''
khipuMeta['DISTINCT_CORD_ATTACH']=''
#khipuMeta['CORDS_HAVE_MULT_COLORS']='' #CORD_COLOR_TYPES has '/'
khipuMeta['MED_CORD_LENGTH']=''
khipuMeta['MAX_CORD_LENGTH']=''
khipuMeta['MIN_CORD_LENGTH']=''
khipuMeta['MED_CORD_VALUE']=''
khipuMeta['MAX_CORD_VALUE']=''
khipuMeta['MIN_CORD_VALUE']=''
khipuMeta['VALUE_SUM']=''
khipuMeta['CORD_COLOR_TYPES']=''
khipuMeta['COUNT_KNOTTED_CORDS']=''
khipuMeta['COUNT_CORDS_NO_KNOTS']=''
khipuMeta['COUNT_BROKEN_CORDS']=''
#monochrome khipus have COUNT_DISTINCT_COLORS == 1

def SetArchive (num):
    inp=''
    if(num==100):
        inp = 'Leymebamba'
    elif(num==200):
        inp = 'Ica'
    elif(num==201):
        inp = 'Circum-Ica'
    elif(num==300):
        inp = 'Pachacamac'
    elif(num==202):
        inp = 'Nazca'
    elif(num==301):
        inp = 'Puruchuco'
   # print(inp)
    return inp
def GetOneResult(string1):
    cur.execute(string1)
    tmp = cur.fetchall()
    try:
        return tmp[0][0]
    except:
        return ''
    
def GetColorType(index,pcordID):
    if(pcordID==0):
        tmpDF = pd.read_sql_query("select * from ascher_cord_color where khipu_id="+str(index),con)
    else:
        tmpDF = pd.read_sql_query("select * from ascher_cord_color where khipu_id="+str(index)+" and cord_id = "+str(pcordID),con)
    tmpStr = ''
    if(tmpDF.FULL_COLOR.str.count(":").sum() !=0):
        tmpStr+=':'
    elif(tmpDF.FULL_COLOR.str.count("%").sum()!=0):
        tmpStr+=':'
    if(tmpDF.FULL_COLOR.str.count("-").sum() !=0):
        tmpStr+='-'
    if(pcordID==0):
        if(GetOneResult("select count(cord_id) from ascher_cord_color where khipu_id="+str(index))>GetOneResult("select count(distinct(cord_id)) from ascher_cord_color where khipu_id="+str(index))):#cords change colors
            tmpStr+="/"
    else:
        if(GetOneResult("select count(cord_id) from ascher_cord_color where khipu_id="+str(index)+" and cord_id = "+str(pcordID))>1):
            tmpStr+='/'
    return tmpStr

rDF = pd.read_sql_query("SELECT khipu_id,banded,seriated FROM quipu_info",con2)
string1 = "SELECT khipu_id FROM quipu_info"
cur2.execute(string1)
idx = cur2.fetchall()
rDF.head()
rDF=rDF.set_index("khipu_id")

#the khipus with investigator_num that starts with Q are incomplete and should be dropped
for index,row in khipuMeta.iterrows():
    try:
        if khipuMeta.at[index,'INVESTIGATOR_NUM'][0]=='Q':
            khipuMeta.drop(index, inplace=True)
    except:
        khipuMeta.drop(index, inplace=True)

        
for index,row in khipuMeta.iterrows():
    
    khipuMeta.at[index,'ARCHIVE']=SetArchive(row['ARCHIVE_NUM'])
    
    string1 = "select * from primary_cord where khipu_id = "+str(index)
    cur.execute(string1)
    tmp = cur.fetchall()
    if(len(tmp)>0):
        pcordID = tmp[0][1]
        khipuMeta.at[index,'PCORD_ID']=pcordID

        khipuMeta.at[index,'PCORD_LENGTH'] = tmp[0][6]
        khipuMeta.at[index,'PCORD_TERM'] = tmp[0][8]
        khipuMeta.at[index,'PRIM_BEG'] = tmp[0][9]
        #print(tmp[0][14])
        khipuMeta.at[index,'PCORD_PLY'] = tmp[0][14]
        khipuMeta.at[index,'PCORD_FULL_COLOR'] = GetOneResult('select FULL_COLOR from ascher_cord_color where khipu_id='+str(index)+' and cord_id = '+str(pcordID))
    
    if(row["REGION"]==''):
        khipuMeta.at[index,'REGION'] = 'unknown'
    if(row["REGION"]==' '):
        khipuMeta.at[index,'REGION'] = 'unknown'
    if(row["REGION"]==None):
        khipuMeta.at[index,'REGION'] = 'unknown'
    khipuMeta.at[index,'PCORD_COLORTYPE'] = GetColorType(index,pcordID)
    khipuMeta.at[index,'CORD_COLOR_TYPES'] = GetColorType(index,0)
    cords = pd.read_sql_query("SELECT * FROM CORD WHERE KHIPU_ID = "+str(index),con)
    
    if(cords.CORD_CLASSIFICATION.str.count("K").sum() ==0):
        khipuMeta.at[index,'PCORD_HAS_KNOTS'] = 0
    else:
        khipuMeta.at[index,'PCORD_HAS_KNOTS'] = 1
    
    if(cords.CORD_CLASSIFICATION.str.count("M").sum() ==0):
        khipuMeta.at[index,'PCORD_HAS_MARKERS'] = 0
    else:
        khipuMeta.at[index,'PCORD_HAS_MARKERS'] = 1
    if(index in idx):
        khipuMeta.at[index,'SERIATED'] = rDF._get_value(index,'seriated')
        khipuMeta.at[index,'BANDED'] = rDF._get_value(index,'banded')
    
    newDF = cords[cords['PENDANT_FROM']== pcordID]
    newDF1 = newDF[newDF['CORD_CLASSIFICATION']!="T"]
    khipuMeta.at[index,'COUNT_PENDANTS'] = len(newDF.index)
    newDF1 = newDF[newDF['CORD_CLASSIFICATION']=="T"]
    khipuMeta.at[index,'COUNT_TOPS'] = len(newDF.index)
    newDF = cords[cords['PENDANT_FROM']!= pcordID]
    khipuMeta.at[index,'COUNT_SUBS'] = len(newDF.index)
    khipuMeta.at[index,'COUNT_DISTINCT_COLORS'] = GetOneResult('SELECT count(distinct(full_color)) FROM ascher_cord_color where khipu_id ='+str(index))
    
    ply = ''
    string1 = "select distinct(twist) from cord where khipu_id = "+str(index)
    cur.execute(string1)
    tmp = cur.fetchall()
    if(len(tmp)>0):
        for t in tmp:
            try:
                ply += t[0]
            except (RuntimeError, TypeError, NameError):
                pass
    khipuMeta.at[index,'DISTINCT_CORD_PLY'] = ply
    
    attach = ''
    string1 = "select distinct(ATTACHMENT_TYPE) from cord where khipu_id = "+str(index)
    cur.execute(string1)
    tmp = cur.fetchall()
    if(len(tmp)>0):
        for t in tmp:
            try:
                attach += t[0]
            except (RuntimeError, TypeError, NameError):
                pass
    khipuMeta.at[index,'DISTINCT_CORD_ATTACH'] = attach
    
    newDF = cords[cords['TERMINATION']!= 'B']
    newDF = cords[cords['CORD_LENGTH']> 0]
    khipuMeta.at[index,'MED_CORD_LENGTH'] = newDF['CORD_LENGTH'].median()
    khipuMeta.at[index,'MAX_CORD_LENGTH'] = newDF['CORD_LENGTH'].max()
    khipuMeta.at[index,'MIN_CORD_LENGTH'] = newDF['CORD_LENGTH'].min()
    
    newDF = cords[cords['cord_value']>0]
    khipuMeta.at[index,'MED_CORD_VALUE'] = newDF['cord_value'].median(skipna=True)
    khipuMeta.at[index,'MAX_CORD_VALUE'] = newDF['cord_value'].max(skipna=True)
    khipuMeta.at[index,'MIN_CORD_VALUE'] = newDF['cord_value'].min(skipna=True)
    newDF = cords[cords['cord_value']>0]
    khipuMeta.at[index,'VALUE_SUM'] = newDF['cord_value'].sum(skipna=True)
    khipuMeta.at[index,'COUNT_KNOTTED_CORDS'] = len(newDF.index)
    
    newDF = cords[cords['cord_value']==0]
    khipuMeta.at[index,'COUNT_CORDS_NO_KNOTS'] = len(newDF.index)
    newDF = cords[cords['TERMINATION']== 'B']
    khipuMeta.at[index,'COUNT_BROKEN_CORDS'] = len(newDF.index)

#dataframe for all cords
cords = pd.read_sql_query("SELECT * FROM CORD", con)
cords.set_index("CORD_ID")

onlyCords = cords[cords['CORD_CLASSIFICATION']!="M"]
onlyCords = onlyCords[onlyCords['CORD_CLASSIFICATION']!="K"]
onlyCords.set_index("CORD_ID",inplace=True)
onlyPendants = onlyCords[onlyCords['CORD_LEVEL']==1]
onlyTop1s = onlyCords[onlyCords['CORD_LEVEL']==(-1)]
onlySubs = onlyCords[onlyCords['CORD_LEVEL']!=1]
onlyBelowSubs = onlyCords[onlyCords['CORD_LEVEL']>1]
onlyTopSubs = onlyCords[onlyCords['CORD_LEVEL']<-1]

column = onlyCords['CORD_LEVEL']
maxLevel = column.max()
for index,row in onlyCords.iterrows():
    if(row['CORD_CLASSIFICATION'] =="T" and row['CORD_LEVEL']>0):
        onlyCords.at[index,'CORD_LEVEL'] = row['CORD_LEVEL'] * -1
onlyTop1s = onlyCords[onlyCords['CORD_LEVEL']==(-1)]

currLevel = 2
for index,row in onlyCords.iterrows():
    if(row['CORD_LEVEL'] == currLevel):
        if(row['PENDANT_FROM'] in onlyTop1s.index):
            onlyCords.at[index,'CORD_LEVEL'] = row['CORD_LEVEL'] * -1

def SetTopSubs(currLevel,onlyTopSubs):
    for index,row in onlyCords.iterrows():
        if(row['CORD_LEVEL'] == currLevel):
            if(row['PENDANT_FROM'] in onlyTopSubs.index):
                onlyCords.at[index,'CORD_LEVEL'] = -currLevel
    onlyTopSubs = onlyCords[onlyCords['CORD_LEVEL']<-1]
        
while currLevel<=maxLevel:
    SetTopSubs(currLevel,onlyTopSubs)
    currLevel +=1

    
onlyPendants = onlyCords[onlyCords['CORD_LEVEL']==1]
onlyTop1s = onlyCords[onlyCords['CORD_LEVEL']==(-1)]
onlySubs = onlyCords[onlyCords['CORD_LEVEL']!=1]
onlyBelowSubs = onlyCords[onlyCords['CORD_LEVEL']>1]
onlyTopSubs = onlyCords[onlyCords['CORD_LEVEL']<-1]

def GetKhipuID2 (value):
    try:
        ID = khipuNamesDF.loc[khipuNamesDF.INVESTIGATOR_NUM == value,'KHIPU_ID'].values[0]
        if ID ==0:
            ID = khipuNamesDF.loc[khipuNamesDF.orig_inv_num == value,'KHIPU_ID'].values[0]
        if ID ==0:
            ID = khipuNamesDF.loc[khipuNamesDF.MUSEUM_NUM == value,'KHIPU_ID'].values[0]
    except:
        try:
           ID = khipuNamesDF.loc[khipuNamesDF.orig_inv_num == value,'KHIPU_ID'].values[0]
        except:
            try:
              ID = khipuNamesDF.loc[khipuNamesDF.MUSEUM_NUM.str.contains(value),'KHIPU_ID'].values[0]
            except:
                ID = 0
    return ID

def GetKhipuID (name):
    name1 = name
    name = name.upper()
    num = 0    
    if len(name)==5:
        num = GetKhipuID2(name)
    elif len(name)==4:
        num = GetKhipuID2(name)
        if num ==0:
            name = name[:2] + "0" + name[2:]
            num = GetKhipuID2(name)
    elif len(name)==3:
        num = GetKhipuID2(name)
        if num ==0:
            name = name[:2] + "00" + name[2:]
            num = GetKhipuID2(name)
    else:
        num = GetKhipuID2(name)
    
    if num == 0:
        print("Can't find {} in df".format(name1))
    return num


#khipuMeta.dropna(subset = ["MAX_CORD_LENGTH"], inplace=True) #remove empty rows - Removed as Ascher did not record cord lengths for cords without knots
print("Ready to search")
khipuMeta.head()



Ready to search


,INVESTIGATOR_NUM,orig_inv_num,PROVENANCE,REGION,ARCHIVE_NUM,MUSEUM_NAME,MUSEUM_NUM,ARCHIVE,PCORD_LENGTH,PCORD_TERM,PCORD_PLY,PCORD_COLORTYPE,PCORD_FULL_COLOR,PCORD_ID,PRIM_BEG,PCORD_HAS_KNOTS,PCORD_HAS_MARKERS,SERIATED,BANDED,COUNT_PENDANTS,COUNT_TOPS,COUNT_SUBS,COUNT_DISTINCT_COLORS,DISTINCT_CORD_PLY,DISTINCT_CORD_ATTACH,MED_CORD_LENGTH,MAX_CORD_LENGTH,MIN_CORD_LENGTH,MED_CORD_VALUE,MAX_CORD_VALUE,MIN_CORD_VALUE,VALUE_SUM,CORD_COLOR_TYPES,COUNT_KNOTTED_CORDS,COUNT_CORDS_NO_KNOTS,COUNT_BROKEN_CORDS
KHIPU_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000000,UR019,UR019,Leymebamba,Chachapoyas,100.0,"Centro Mallqui, Leymebamba, Amazonas",CMA-480/LC1-109.3,Leymebamba,26.0,K,S,,W,1000000,K,0,0,,,13,13,5,7,S,UVR,31.0,49.5,0.5,8.5,28,1,100,:,10,8,3
1000001,UR008,UR008,Leymebamba,Chachapoyas,100.0,"Centro Mallqui, Leymebamba, Amazonas",CMA-480/LC1-109.5,Leymebamba,16.5,K,S,,W,1000001,T,0,0,,,6,6,0,1,S,V,26.25,29.0,15.5,83.5,200,5,502,,6,0,0
1000002,UR020,UR020,Leymebamba,Chachapoyas,100.0,"Centro Mallqui, Leymebamba, Amazonas",CMA-480/LC1-109.6,Leymebamba,10.5,,S,:,KB:AB,1000002,,0,1,,,24,24,6,14,S,RVU,36.75,52.0,14.0,354.0,5004.0,2.0,25965.0,:-,28,0,0
1000003,UR018,UR018,Leymebamba,Chachapoyas,100.0,"Centro Mallqui, Leymebamba, Amazonas",CMA-480/LC1-109B,Leymebamba,98.0,K,S,,W,1000003,K,1,0,,,31,31,7,9,S,RVU,36.5,58.0,15.0,50.5,9018.0,1.0,27563.0,:/,32,2,0
1000004,UR010,UR010,Leymebamba,Chachapoyas,100.0,"Centro Mallqui, Leymebamba, Amazonas",CMA-480/LC1-109.1,Leymebamba,65.5,K,S,,W,1000004,T,1,0,,,159,159,17,23,S,VRU,38.0,63.0,1.0,12.0,153.0,1.0,2997.0,:-/,158,16,7


In [2]:
#data cleaning to handle null or empty values
def ReplaceNull(df, RowName, NullValue):
    for index,row in df.iterrows():
        if row[RowName] is None:
            row[RowName]=NullValue
ReplaceNull(khipuMeta, 'PROVENANCE','unknown')
ReplaceNull(khipuMeta, 'REGION','unknown')
ReplaceNull(khipuMeta, 'PCORD_TERM','')
ReplaceNull(khipuMeta, 'PCORD_PLY','U')
ReplaceNull(khipuMeta, 'PRIM_BEG','')


# Explanation of data frames and definitions
### Data frames
<br><b>khipuNamesDF</b> is a dataframe for finding the ID number of a particular khipu
<br><b>khipuMeta</b> has data for all khipu, including primary cord, how many pendants, max and min cord values, etc. Indexed by the KHIPU_ID.
<br><b>onlyCords</b> contains all of the cords for all of the khipus for comparisons across the database. <b>onlyPendants</b>, <b>onlyTop1s</b>, <b>onlySubs</b>, <b>onlyBelowSubs</b>, <b>onlyTopSubs</b> are the same as <b>onlyCords</b>, but a limited subset.

### Definitions
<br><b>GetKhipuID()</b> can be used to look up a khipu ID by investigator name or museum designation. Example to look up the khipu ID of AS20: <b>GetKhipuID('AS20')</b>
<br>For a SQL query of the database with 1 result use <b>GetOneResult('query')</b>


In [3]:
print('There are {} khipus included in this dataset'.format(len(khipuMeta.index)))
print('with a total of {} cords ({} pendants, {} top cords and {} subsidiary cords).'.format(len(onlyCords.index),len(onlyPendants.index),len(onlyTop1s.index),len(onlySubs.index)))
print()
print ('Khipu_ID for AS20 is {}'.format(GetKhipuID('AS20')))
print(GetOneResult('select FULL_COLOR from ascher_cord_color where khipu_id=1000321 and cord_id = 1000312'))

There are 626 khipus included in this dataset
with a total of 56947 cords (39889 pendants, 274 top cords and 17058 subsidiary cords).

Khipu_ID for AS20 is 1000191
AB:KB:MB


In [4]:
khipuMeta.head()

,INVESTIGATOR_NUM,orig_inv_num,PROVENANCE,REGION,ARCHIVE_NUM,MUSEUM_NAME,MUSEUM_NUM,ARCHIVE,PCORD_LENGTH,PCORD_TERM,PCORD_PLY,PCORD_COLORTYPE,PCORD_FULL_COLOR,PCORD_ID,PRIM_BEG,PCORD_HAS_KNOTS,PCORD_HAS_MARKERS,SERIATED,BANDED,COUNT_PENDANTS,COUNT_TOPS,COUNT_SUBS,COUNT_DISTINCT_COLORS,DISTINCT_CORD_PLY,DISTINCT_CORD_ATTACH,MED_CORD_LENGTH,MAX_CORD_LENGTH,MIN_CORD_LENGTH,MED_CORD_VALUE,MAX_CORD_VALUE,MIN_CORD_VALUE,VALUE_SUM,CORD_COLOR_TYPES,COUNT_KNOTTED_CORDS,COUNT_CORDS_NO_KNOTS,COUNT_BROKEN_CORDS
KHIPU_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000000,UR019,UR019,Leymebamba,Chachapoyas,100.0,"Centro Mallqui, Leymebamba, Amazonas",CMA-480/LC1-109.3,Leymebamba,26.0,K,S,,W,1000000,K,0,0,,,13,13,5,7,S,UVR,31.0,49.5,0.5,8.5,28,1,100,:,10,8,3
1000001,UR008,UR008,Leymebamba,Chachapoyas,100.0,"Centro Mallqui, Leymebamba, Amazonas",CMA-480/LC1-109.5,Leymebamba,16.5,K,S,,W,1000001,T,0,0,,,6,6,0,1,S,V,26.25,29.0,15.5,83.5,200,5,502,,6,0,0
1000002,UR020,UR020,Leymebamba,Chachapoyas,100.0,"Centro Mallqui, Leymebamba, Amazonas",CMA-480/LC1-109.6,Leymebamba,10.5,,S,:,KB:AB,1000002,,0,1,,,24,24,6,14,S,RVU,36.75,52.0,14.0,354.0,5004.0,2.0,25965.0,:-,28,0,0
1000003,UR018,UR018,Leymebamba,Chachapoyas,100.0,"Centro Mallqui, Leymebamba, Amazonas",CMA-480/LC1-109B,Leymebamba,98.0,K,S,,W,1000003,K,1,0,,,31,31,7,9,S,RVU,36.5,58.0,15.0,50.5,9018.0,1.0,27563.0,:/,32,2,0
1000004,UR010,UR010,Leymebamba,Chachapoyas,100.0,"Centro Mallqui, Leymebamba, Amazonas",CMA-480/LC1-109.1,Leymebamba,65.5,K,S,,W,1000004,T,1,0,,,159,159,17,23,S,VRU,38.0,63.0,1.0,12.0,153.0,1.0,2997.0,:-/,158,16,7


In [5]:
onlyCords.head()

,KHIPU_ID,TWIST_ANGLE,THICKNESS,FIBER,TERMINATION,PENDANT_FROM,ATTACHED_TO,ATTACHMENT_TYPE,CLUSTER_ID,CLUSTER_ORDINAL,CORD_ORDINAL,ATTACH_POS,CORD_LENGTH,CREATED_BY,CREATED_ON,CHANGED_BY,CHANGED_ON,CORD_LEVEL,CORD_NOTES,TWIST,CORD_CLASSIFICATION,INVESTIGATOR_CORD_NUM,CANUTO_ORDINAL,CANUTO_ID,cord_value
CORD_ID,,,,,,,,,,,,,,,,,,,,,,,,,
3000001,1000000,0.0,0.0,CN,B,1000000,0.0,U,1000000,1,1,0.0,0.5,cbrezine,2011-12-23 17:44:41,gurton,2007-12-12 20:44:38,1,,S,,1,0.0,0.0,0.0
3000002,1000000,0.0,0.0,CN,B,1000000,0.0,V,1000000,2,2,0.0,16.0,cbrezine,2011-12-23 17:44:41,cbrezine,2003-05-28 21:14:22,1,,S,,2,0.0,0.0,0.0
3000003,1000000,0.0,0.0,CN,K,1000000,0.0,V,1000000,3,3,0.0,32.0,cbrezine,2011-12-23 17:44:41,cbrezine,2003-05-28 21:14:40,1,,S,,3,0.0,0.0,18.0
3000004,1000000,0.0,0.0,CN,U,1000000,0.0,V,1000001,1,4,0.0,39.5,cbrezine,2011-12-23 17:44:41,cbrezine,2003-05-28 21:16:33,1,,S,,4,0.0,0.0,1.0
3000005,1000000,0.0,0.0,CN,B,1000000,0.0,R,1000001,2,5,0.0,21.0,cbrezine,2011-12-23 17:44:41,cbrezine,2003-05-28 21:26:28,1,,S,,5,0.0,0.0,0.0


In [6]:
#pivot tables
cord_value_pivot = khipuMeta.pivot_table('MED_CORD_VALUE',columns = 'KHIPU_ID',)

DataError: No numeric types to aggregate

## Primary Cords
### Primary cord information is saved in the <b>khipuMeta</b> dataframe

In [ ]:
df = khipuMeta.copy()

print('Of the {} khipus listed in the database, '.format(len(df.index)))
for index, row in df.iterrows():
    try:
        x = row['PCORD_LENGTH'] +5
        if x ==5:
            print('{}: {} primary cord has a length of {}'.format(index, row['orig_inv_num'],row['PCORD_LENGTH']))
            df.drop(index, inplace=True)

    except:
        print('{}: {} primary cord has a length of {}'.format(index, row['orig_inv_num'],row['PCORD_LENGTH']))
        df.drop(index, inplace=True)

print('{} have a primary cord'.format(len(df.index)))

In [ ]:
colors = sns.color_palette('pastel')[0:2]
labels = ['Has a primary cord','No primary cord']
values = [len(df.index),len(khipuMeta.index)-len(df.index)]
explode = [0, 0.1]  # only "explode" the 2nd slice

plt.pie(values, labels=labels, colors = colors, explode = explode)
plt.savefig('HasPrimCord',dpi=200)
plt.show()


In [ ]:
print('Max: {} cm'.format(df['PCORD_LENGTH'].max()))
print('Min: {} cm'.format(df['PCORD_LENGTH'].min()))
print('Average: {} cm'.format(df['PCORD_LENGTH'].mean()))

plt.boxplot(df['PCORD_LENGTH'])

In [ ]:
plt.scatter(x=df.index,y=df['PCORD_LENGTH'])
plt.savefig('PCordScatter',dpi=200)


In [ ]:
for index, row in df.iterrows():
    if row['PCORD_LENGTH']>250:
        print(str(row['orig_inv_num'])+': '+str(row['PCORD_LENGTH'])+' cm')

In [ ]:
df2 = df.copy()
for index, row in df2.iterrows():
    try:
        if 'B' in row['PCORD_TERM']:
            df2.drop(index, inplace=True)
    except:
        df2.at[index,'PCORD_TERM']=''
    try:
        if 'B' in row['PRIM_BEG']:
            df2.drop(index, inplace=True)
    except:
        df2.at[index,'PRIM_BEG']=''
print(len(df2.index))

In [ ]:
print('Max: {} cm'.format(df['PCORD_LENGTH'].max()))
print('Min: {} cm'.format(df['PCORD_LENGTH'].min()))
print('Average: {} cm'.format(df['PCORD_LENGTH'].mean()))

print('Max: {} cm'.format(df2['PCORD_LENGTH'].max()))
print('Min: {} cm'.format(df2['PCORD_LENGTH'].min()))
print('Average: {} cm'.format(df2['PCORD_LENGTH'].mean()))


In [ ]:
for index, row in df2.iterrows():
    if row['PCORD_LENGTH'] >100:
        df2.drop(index, inplace=True)
        
plt.scatter(x=df2.index,y=df2['PCORD_LENGTH'])

In [ ]:
labels =['Knotted','Broken','Ravelled','Doubled','Cut','Wooden Bar']
k=0
b=0
r=0
d=0
c=0
wb=0

for index,row in khipuMeta.iterrows():
    try:
        if 'K' in row['PCORD_TERM']:
            k+=1
        elif 'WB' in row['PCORD_TERM']:
            wb+=1
        elif 'B' in row['PCORD_TERM']:
            b+=1
        elif 'R' in row['PCORD_TERM']:
            r+=1
        elif 'U' in row['PCORD_TERM']:
            r+=1
        elif 'C' in row['PCORD_TERM']:
            c+=1
        elif 'D' in row['PCORD_TERM']:
            d+=1
        else:
            r+=1
        
        
    except:
        r+=1


values = [k,b,r,d,c,wb]
print('K: {}, B: {}, R: {}, D: {}, C: {}, WB: {}'.format(k,b,r,d,c,wb))
plt.pie(values, labels=labels)
plt.show()

In [ ]:
#beginning
labels =['Knotted','Broken','Ravelled','Twisted','Not listed','Woven Cord Ball','Woven Bundle','Attached to Other Khipu','Tassel/Borla','Needlework Bundle']
k=0
b=0
r=0
t=0
nl=0
wcb=0
wb=0
x=0
tl=0
nb=0

for index,row in khipuMeta.iterrows():
    try:
        if 'K' in row['PRIM_BEG']:
            k+=1
        elif 'woven cord ball' in row['PRIM_BEG']:
            wcb+=1
        elif 'WB' in row['PRIM_BEG']:
            wb+=1
        elif 'NB' in row['PRIM_BEG']:
            nb+=1
        elif 'B' in row['PRIM_BEG']:
            b+=1
        elif 'R' in row['PRIM_BEG']:
            r+=1
        elif 'TL' in row['PRIM_BEG']:
            tl+=1
        elif 'T' in row['PRIM_BEG']:
            t+=1
        elif 'X' in row['PRIM_BEG']:
            x+=1
        else:
            nl+=1
        
        
    except:
        nl+=1

values = [k,b,r,t,nl,wcb,wb,x,tl,nb]
print('{},{},{},{},{},{},{},{},{},{}'.format(k,b,r,t,nl,wcb,wb,x,tl,nb))
plt.pie(values, labels=labels)
plt.show()

In [ ]:
#ply/twist
labels =['S','Z','U']
s=0
z=0
u=0

for index,row in khipuMeta.iterrows():
    try:
        if 'S' in row['PCORD_PLY']:
            s+=1
        elif 'Z' in row['PCORD_PLY']:
            z+=1
        elif 'U' in row['PCORD_PLY']:
            u+=1
        else:
            u+=1
        
        
    except:
        u+=1


values = [s,z,u]
print('S: {}, Z: {}, U: {}'.format(s,z,u))
plt.pie(values, labels=labels)
plt.show()

In [ ]:
print(list(df))

In [ ]:
#compare primary cord length against other attributes

df2 = df.copy() #this will only have primary cords
df2.drop(columns =['INVESTIGATOR_NUM', 'orig_inv_num', 'PROVENANCE', 'REGION', 'ARCHIVE_NUM', 'MUSEUM_NAME', 'MUSEUM_NUM', 
         'ARCHIVE', 'PCORD_TERM', 'PCORD_PLY', 'PCORD_COLORTYPE', 'PCORD_FULL_COLOR', 'PCORD_ID', 
         'PRIM_BEG', 'PCORD_HAS_KNOTS', 'PCORD_HAS_MARKERS', 'SERIATED', 'BANDED', 'COUNT_TOPS', 
         'COUNT_SUBS', 'COUNT_DISTINCT_COLORS', 'DISTINCT_CORD_PLY', 'DISTINCT_CORD_ATTACH', 'MED_CORD_LENGTH', 
         'MAX_CORD_LENGTH', 'MIN_CORD_LENGTH', 'MED_CORD_VALUE', 'MAX_CORD_VALUE', 'MIN_CORD_VALUE', 'VALUE_SUM', 
         'CORD_COLOR_TYPES', 'COUNT_KNOTTED_CORDS', 'COUNT_CORDS_NO_KNOTS', 'COUNT_BROKEN_CORDS'])


sns.pairplot(df, hue="PCORD_LENGTH")

In [ ]:
df2 = df.copy() #this will only have primary cords
df2.drop['INVESTIGATOR_NUM', 'orig_inv_num', 'PROVENANCE', 'REGION', 'ARCHIVE_NUM', 'MUSEUM_NAME', 'MUSEUM_NUM', 
         'ARCHIVE', 'PCORD_LENGTH', 'PCORD_TERM', 'PCORD_PLY', 'PCORD_COLORTYPE', 'PCORD_FULL_COLOR', 'PCORD_ID', 
         'PRIM_BEG', 'PCORD_HAS_KNOTS', 'PCORD_HAS_MARKERS', 'SERIATED', 'BANDED', 'COUNT_PENDANTS', 'COUNT_TOPS', 
         'COUNT_SUBS', 'COUNT_DISTINCT_COLORS', 'DISTINCT_CORD_PLY', 'DISTINCT_CORD_ATTACH', 'MED_CORD_LENGTH', 
         'MAX_CORD_LENGTH', 'MIN_CORD_LENGTH', 'MED_CORD_VALUE', 'MAX_CORD_VALUE', 'MIN_CORD_VALUE', 'VALUE_SUM', 
         'CORD_COLOR_TYPES', 'COUNT_KNOTTED_CORDS', 'COUNT_CORDS_NO_KNOTS', 'COUNT_BROKEN_CORDS']

df2.dropna('ARCHIVE_NUM')
sns.pairplot(df, hue="PCORD_LENGTH")

In [ ]:

df2 = df.copy()
for index, row in df2.iterrows():
    try:
        #add something to test if cell data is a number
        if 'B' in row['PCORD_TERM']:
            df2.drop(index, inplace=True)
        elif 'K' in row['PCORD_TERM']:
            df2.drop(index, inplace=True)
        elif 'D' in row['PCORD_TERM']:
            df2.drop(index, inplace=True)
        elif 'C' in row['PCORD_TERM']:
            df2.drop(index, inplace=True)
    except:
        df2.at[index,'PCORD_TERM'] = 'none'

for index, row in df2.iterrows():
    try:
        #add something to test if cell data is a number
        if 'B' in row['PCORD_LENGTH']:
            df2.drop(index, inplace=True)
        elif 'K' in row['PCORD_LENGTH']:
            df2.drop(index, inplace=True)
        elif 'D' in row['PCORD_LENGTH']:
            df2.drop(index, inplace=True)
        elif 'C' in row['PCORD_LENGTH']:
            df2.drop(index, inplace=True)
    except:
        df2.at[index,'PCORD_TERM'] = 'none'
      
df2 = df[df['PCORD_TERM'].str.contains('B')]
plt.scatter(df2.index,df2['PCORD_LENGTH'], c='red',label = 'Broken')
df2 = df[df['PCORD_TERM'].str.contains('K')]
plt.scatter(df2.index,df2['PCORD_LENGTH'], c='blue',label = 'Knotted')


plt.scatter(df2.index,df2['PCORD_LENGTH'], c='orange',label = 'Raveled')
df2 = df[df['PCORD_TERM'].str.contains('D')]
plt.scatter(df2.index,df2['PCORD_LENGTH'], c='purple',label = 'Doubled')
df2 = df[df['PCORD_TERM'].str.contains('C')]
plt.scatter(df2.index,df2['PCORD_LENGTH'], c='green',label = 'Cut')
plt.legend()
plt.show()


In [ ]:
#tail length
df['TAIL_LENGTH']=''
for index,row in df.iterrows():
    query = 'select spacing from cord_cluster where khipu_ID = '+str(index)+' and cord_id = (select pcord_id from primary_cord where khipu_ID = '+str(index)+') and ordinal = (select max(ordinal) from cord_cluster where khipu_ID = '+str(index)+' and cord_id = (select pcord_id from primary_cord where khipu_ID = '+str(index)+'))'
    df.at[index,row['TAIL_LENGTH']]=GetOneResult(query)
df.head()


In [ ]:
#take out the broken cords to get an idea of the true length
df2 = df.copy()
for index, row in df2.iterrows():

    try:
        #add something to test if cell data is a number
        if 'B' in row['TAIL_LENGTH']:
            df2.drop(index, inplace=True)
    except:
        df2.at[index,'TAIL_LENGTH'] = ''
        
print('Max: {}'.format(df['TAIL_LENGTH'].max()))
print('Min: {}'.format(df['TAIL_LENGTH'].min()))
print('Average: {}'.format(df['TAIL_LENGTH'].mean()))

print('Max: {}'.format(df2['TAIL_LENGTH'].max()))
print('Min: {}'.format(df2['TAIL_LENGTH'].min()))
print('Average: {}'.format(df2['TAIL_LENGTH'].mean()))
plt.scatter(y=df['TAIL_LENGTH'],x=df.index)


In [ ]:
sns.catplot(data=cords,x='KHIPU_ID',y='cord_value',kind='box')

## Make graphs

In [ ]:
khipuMetaNum = khipuMeta.copy()
khipuMetaNum["VALUE_SUM"] = pd.to_numeric(khipuMetaNum["VALUE_SUM"], downcast="float")
khipuMetaNum["COUNT_KNOTTED_CORDS"] = pd.to_numeric(khipuMetaNum["COUNT_KNOTTED_CORDS"], downcast="float")
khipuMetaNum["PCORD_LENGTH"] = pd.to_numeric(khipuMetaNum["PCORD_LENGTH"], downcast="float")
khipuMetaNum["COUNT_PENDANTS"] = pd.to_numeric(khipuMetaNum["COUNT_PENDANTS"], downcast="float")
khipuMetaNum["COUNT_TOPS"] = pd.to_numeric(khipuMetaNum["COUNT_TOPS"], downcast="float")
khipuMetaNum["COUNT_SUBS"] = pd.to_numeric(khipuMetaNum["COUNT_SUBS"], downcast="float")
khipuMetaNum["COUNT_DISTINCT_COLORS"] = pd.to_numeric(khipuMetaNum["COUNT_DISTINCT_COLORS"], downcast="float")
khipuMetaNum["MAX_CORD_LENGTH"] = pd.to_numeric(khipuMetaNum["MAX_CORD_LENGTH"], downcast="float")
khipuMetaNum["MIN_CORD_LENGTH"] = pd.to_numeric(khipuMetaNum["MIN_CORD_LENGTH"], downcast="float")
khipuMetaNum["MED_CORD_VALUE"] = pd.to_numeric(khipuMetaNum["MED_CORD_VALUE"], downcast="float")
khipuMetaNum["MAX_CORD_VALUE"] = pd.to_numeric(khipuMetaNum["MAX_CORD_VALUE"], downcast="float")
khipuMetaNum["MIN_CORD_VALUE"] = pd.to_numeric(khipuMetaNum["MIN_CORD_VALUE"], downcast="float")
khipuMetaNum["COUNT_CORDS_NO_KNOTS"] = pd.to_numeric(khipuMetaNum["COUNT_CORDS_NO_KNOTS"], downcast="float")
khipuMetaNum["COUNT_BROKEN_CORDS"] = pd.to_numeric(khipuMetaNum["COUNT_BROKEN_CORDS"], downcast="float")
khipuMetaNum.describe()


### A look at knots and knotted cords

In [ ]:
print("Of the {} khipus recorded in this database, {} have no knots on any cord.".format(len(khipuMetaNum.index),len(khipuMetaNum[khipuMetaNum['VALUE_SUM']==0])))
p = sns.histplot(data = khipuMetaNum,x="VALUE_SUM", bins=50)
p.set_xlabel('Sum of all the cord values for each khipu')
p.set(title='All Khipus')
plt2.savefig('all_sums.png')
khipuMetaNum.VALUE_SUM.describe()

In [ ]:
#ahead of the lineplot, make only numbers
#onlyPendants["INVESTIGATOR_CORD_NUM"] = pd.to_numeric(onlyPendants["INVESTIGATOR_CORD_NUM"], downcast="float")
#onlyPendants["cord_value"] = pd.to_numeric(onlyPendants["cord_value"], downcast="float")
for index, row in onlyPendants.iterrows():
    try:
        row['cord_value'] + 5
    except (ValueError, TypeError):
        try:
            row['cord_value'] = float(row['cord_value'])
        except:
            row['cord_value'] = 0
        


def MakePlot(num):
    df = onlyPendants[onlyPendants['KHIPU_ID']==num]
    df = df[['INVESTIGATOR_CORD_NUM','cord_value']].copy()
    df.dropna()
    #print("Of the {} khipus recorded in the ".format(len(df.index))+archive+" archive, {} have no knots on any cord.".format(len(df[df['VALUE_SUM']==0])))
    print(num)
    print(df.head())
    
    p = sns.lineplot(x='INVESTIGATOR_CORD_NUM',y='cord_value', data = df)
    p.set_xlabel('Cord Ordinal')
    quipuName = index
    try:
        quipuName = khipuMeta.loc[num,"orig_inv_num"]
    except:
        try:
            quipuName = khipuMeta.loc[num,"INVESTIGATOR_NUM"]
        except:
            print('Error in naming {}'.format(num))
     
    p.set(title='Khipu {}'.format(quipuName))
    plt2.savefig('value lineplot_'+str(num)+'.png')
    
    #clear figure, otherwise the plot accumulates multiple khipus
    plt2.clf()
    
for index, row in khipuMeta.iterrows():
    #make the plot for each khipu
    MakePlot(index)
    
#cords.head()

In [ ]:
def MakePlot(archive):
    df = khipuMetaNum[khipuMetaNum['ARCHIVE']==archive]
    print("Of the {} khipus recorded in the ".format(len(df.index))+archive+" archive, {} have no knots on any cord.".format(len(df[df['VALUE_SUM']==0])))
    p = sns.histplot(data = df,x="VALUE_SUM")
    p.set_xlabel('Sum of all the cord values for each khipu')
    p.set(title='{} Archive'.format(archive))
    plt2.savefig('sums_'+archive+'.png')
archives = ['Leymebamba','Ica','Nazca', 'Pachacamac', 'Circum-Ica', 'Puruchuco']

MakePlot('Puruchuco')


In [ ]:
top75 = khipuMetaNum[khipuMetaNum['VALUE_SUM']>4176]
print(len(top75.index))
df = khipuMetaNum[khipuMetaNum['VALUE_SUM']<4176]

In [ ]:
#TODO - x=number of cords y=value_sum, but also check against primary cord
p = sns.histplot(data = top75,x="VALUE_SUM")
p.set_xlabel('Sum of all the cord values for each khipu')
p.set(title='{} Archive'.format(archive))
plt2.savefig('sums_'+archive+'.png')

In [ ]:
cordValue = cords[cords.cord_value != 0]
onlyCords.describe()
#sns.distplot(cords['cord_value'],kde=False)

In [ ]:
df = khipuMeta.copy()
print(df.dtypes)
df["VALUE_SUM"] = pd.to_numeric(df["VALUE_SUM"], downcast="float")
df["COUNT_KNOTTED_CORDS"] = pd.to_numeric(df["COUNT_KNOTTED_CORDS"], downcast="float")

print(df.dtypes)
sns.set(style='darkgrid')
sns.regplot(x=khipuNum['COUNT_KNOTTED_CORDS'], y=khipuNum['VALUE_SUM'])

In [ ]:
for index,row in cords.iterrows():
    if(row['CORD_CLASSIFICATION']=="T"):
        if(row['CORD_LEVEL']>0):
            cords.at[index,'COUNT_CORDS_NO_KNOTS'] = len(newDF.index)

In [ ]:
cords = pd.read_sql_query("SELECT * FROM CORD WHERE KHIPU_ID = "+str(qID),con)
sns.displot(cords['cord_value'])
cords_onlyPendants = cords[cords['PENDANT_FROM']== pcordID]
cords_onlyTops = cords_onlyPendants[cords_onlyPendants['CORD_CLASSIFICATION']=="T"]
cords_onlyPendants = cords_onlyPendants[cords_onlyPendants['CORD_CLASSIFICATION']!="T"]
cords_onlySubs = cords[cords['PENDANT_FROM']!= pcordID]
sns.displot(cords['cord_value'])
sns.displot(cords_onlyPendants['cord_value'])
sns.displot(cords_onlyTops['cord_value'])
sns.displot(cords_onlySubs['cord_value'])
cords.head()

In [ ]:
sns.jointplot(x='CORD_LEVEL',y='cord_value',data=cords, kind = 'reg')
sns.jointplot(x='INVESTIGATOR_CORD_NUM',y='cord_value',data=cords_onlyPendants, kind = 'reg')
sns.jointplot( x='INVESTIGATOR_CORD_NUM',y='cord_value',data=cords_onlyTops, kind = 'reg')
sns.jointplot( x='INVESTIGATOR_CORD_NUM',y='cord_value',data=cords_onlySubs, kind = 'reg')
